# fetch_20newsgroupsのデータセットを使って文章分類

In [4]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.preprocessing import StandardScaler 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [5]:
NewsGroup = fetch_20newsgroups()

In [13]:
data = NewsGroup.data
target = NewsGroup.target
target_names = NewsGroup.target_names

In [11]:
len(data)

11314

In [12]:
len(target)

11314

In [14]:
target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']